In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import glob
import json
import time
import re
import csv
import PyPDF2
import requests
from dotenv import load_dotenv
load_dotenv()
API_KEY  = os.getenv("GEMINI_API_KEY")
MODEL_ID = "gemini-2.5-flash"
ENDPOINT = (
    f"https://generativelanguage.googleapis.com/v1beta/models/"
    f"{MODEL_ID}:generateContent?key={API_KEY}"
)

# PDF들이 들어있는 폴더
PDF_DIR = os.path.join(os.getcwd(), "downloads")
# 결과 CSV 파일 경로
CSV_PATH = "policy_actions.csv"

# downloads/*.pdf 전부 가져오기
pdf_files = glob.glob(os.path.join(PDF_DIR, "*.pdf"))
print(f"[LOG] 처리할 PDF 파일 {len(pdf_files)}개 발견")

# CSV 파일 열어서 헤더 쓰기
with open(CSV_PATH, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["정책명", "내용"])  # 두 개의 컬럼

    for pdf_path in pdf_files:
        print(f"[LOG] === {os.path.basename(pdf_path)} 처리 시작 ===")

        # PDF 텍스트 추출 (앞 300자만)
        with open(pdf_path, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            text = "\n".join(page.extract_text() or "" for page in reader.pages)
        excerpt = text[:500] + ("\n…이하생략…" if len(text) > 400 else "")

        # Gemini 호출용 프롬프트
        prompt = f"""
아래는 PDF 문서의 일부입니다. 이 부분에서 ‘정책명’과
‘일반 소비자(개인)에게 요구하는 구체적 행동’을
줄글이 아닌, 반드시 다섯(5)개의 독립된 지침으로
번호를 매겨(list) 출력해 주세요.
친환경 행동 위주로, 동사형 문장으로 간결하게 작성하시고,
각 항목 끝에는 마침표를 붙여 주세요.

PDF 내용 일부:
{excerpt}
"""

        payload = {
            "contents": [
                {"parts": [{"text": prompt}]}
            ],
            "generationConfig": {
                "temperature": 0.0,
                "maxOutputTokens": 1024,
                "thinkingConfig": {"thinkingBudget": 0}
            }
        }
        body = json.dumps(payload, ensure_ascii=False).encode("utf-8")
        headers = {"Content-Type": "application/json; charset=utf-8"}

        # API 호출
        resp = requests.post(ENDPOINT, headers=headers, data=body)
        resp.raise_for_status()
        result = resp.json()["candidates"][0]["content"]["parts"][0]["text"]
        print("[LOG] 모델 응답 수신")

        # 응답에서 '정책명' 추출
        m = re.search(r'정책명[:：]\s*(.+)', result)
        policy_name = m.group(1).strip() if m else "UNKNOWN"

        # 응답에서 '행동' 항목 추출
        actions = re.findall(r'\d+\.\s*(.+?)[\.。\n]', result)
        if not actions:
            actions = re.findall(r'\d+\.\s*(.+)', result)

        # CSV에 저장 (정책명, 내용)
        for action in actions:
            line = action.strip()
            if not re.search(r'[\.。]$', line):
                line += "。"
            writer.writerow([policy_name, line])
        print(f"[LOG] '{policy_name}' → {len(actions)}개 항목 기록")

        # 요청 간 딜레이
        time.sleep(1)




[LOG] 처리할 PDF 파일 12개 발견
[LOG] === GreenWave정기예금설명서210526 (1).pdf 처리 시작 ===
[LOG] 모델 응답 수신
[LOG] 'KB Green Wave 1.5 ℃ 정기예금' → 6개 항목 기록
[LOG] === GreenWave정기예금설명서210526.pdf 처리 시작 ===
[LOG] 모델 응답 수신
[LOG] '** KB Green Wave 1.5 ℃ 정기예금' → 6개 항목 기록
[LOG] === KBGreenWave정기예금특약2021924 (1).pdf 처리 시작 ===
[LOG] 모델 응답 수신
[LOG] 'KB Green Wave 1.5 ℃ 정기예금' → 6개 항목 기록
[LOG] === KBGreenWave정기예금특약2021924.pdf 처리 시작 ===
[LOG] 모델 응답 수신
[LOG] 'KB Green Wave 1.5 ℃ 정기예금' → 6개 항목 기록
[LOG] === KBGreenWave정기예금특약210526 (1).pdf 처리 시작 ===
[LOG] 모델 응답 수신
[LOG] 'KB Green Wave 1.5 ℃ 정기예금' → 6개 항목 기록
[LOG] === KBGreenWave정기예금특약210526.pdf 처리 시작 ===
[LOG] 모델 응답 수신
[LOG] 'KB Green Wave 1.5 ℃ 정기예금' → 6개 항목 기록
[LOG] === KB맑은바다적금특약 (1).pdf 처리 시작 ===
[LOG] 모델 응답 수신
[LOG] '** KB맑은바다적금' → 5개 항목 기록
[LOG] === KB맑은바다적금특약.pdf 처리 시작 ===
[LOG] 모델 응답 수신
[LOG] '** KB맑은바다적금' → 5개 항목 기록
[LOG] === 맑은바다적금특약220308 (1).pdf 처리 시작 ===
[LOG] 모델 응답 수신
[LOG] '** KB맑은바다적금' → 5개 항목 기록
[LOG] === 맑은바다적금특약220308.pdf 처리 시작 ===
[LOG] 모델 응답 수신
[LOG] '** 

In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import json
import PyPDF2
import requests
from dotenv import load_dotenv
load_dotenv()
API_KEY  = os.getenv("GEMINI_API_KEY")
MODEL_ID = "gemini-2.5-flash"
ENDPOINT = (
    f"https://generativelanguage.googleapis.com/v1beta/models/"
    f"{MODEL_ID}:generateContent?key={API_KEY}"
)


# 프롬프트
prompt = f"""
아래 소비 현황을 시계열 요소를 반영해 증감 현황을 비교하고 및 고소비 저소비 내용을 1~2문장으로 간결히 요약하고, 탄소 감축을 위한 동사형 액션 플랜 반드시 다섯(5)가지를 제시해 주세요.

월별 계산:1등, 주별:3등, 일별:5등 (등수↑=탄소 소비↓)
고소비: 요식업, 카페, 교통업
저소비: 생활 가전

{excerpt}
"""

payload = {
    "contents": [
        {"parts": [{"text": prompt}]}
    ],
    "generationConfig": {
        "temperature": 0.0,
        "maxOutputTokens": 512,
        "thinkingConfig": {
            "thinkingBudget": 0
        }
    }
}

body = json.dumps(payload, ensure_ascii=False).encode("utf-8")
headers = {"Content-Type": "application/json; charset=utf-8"}

# 호출
resp = requests.post(ENDPOINT, headers=headers, data=body)
print("HTTP", resp.status_code)
print("raw resp:", resp.text[:500], "…")
resp.raise_for_status()

data = resp.json()
result = data["candidates"][0]["content"]["parts"][0]["text"]

print(result or "[응답이 비어있습니다]")
output_path = "gemini_customer_response.txt"
with open(output_path, "w", encoding="utf-8") as f:
    f.write(result or "")


HTTP 200
raw resp: {
  "candidates": [
    {
      "content": {
        "parts": [
          {
            "text": "## 소비 현황 분석 및 탄소 감축 액션 플랜\n\n### 1. 소비 증감 현황 (시계열 요소 반영)\n\n월별 탄소 소비는 가장 낮고(1등), 주별 탄소 소비는 중간이며(3등), 일별 탄소 소비는 가장 높습니다(5등). 이는 **단기적으로는 탄소 소비가 높지만, 장기적으로는 탄소 소비를 줄이는 경향**을 보인다고 해석할 수 있습니다. 즉, 일상적인 소비 습관에서는 탄소 배출이 많지만, 월 단위로 계획하거나 의식적으로 소비할 때는 탄소 배출을 줄이려는 노력이 반영되고 있음을 시사합니다.\n\n### 2. 고소비 및 저소비 요약\n\n*   **고소비:** 요식업, 카페, 교통업은 탄소 배출량이 높은 소비 영역입니다.\n*   **저소비:** 생활 가전은 탄소 배출량이 낮은 소비 영역입니다.\n\n### 3. 탄소 …
## 소비 현황 분석 및 탄소 감축 액션 플랜

### 1. 소비 증감 현황 (시계열 요소 반영)

월별 탄소 소비는 가장 낮고(1등), 주별 탄소 소비는 중간이며(3등), 일별 탄소 소비는 가장 높습니다(5등). 이는 **단기적으로는 탄소 소비가 높지만, 장기적으로는 탄소 소비를 줄이는 경향**을 보인다고 해석할 수 있습니다. 즉, 일상적인 소비 습관에서는 탄소 배출이 많지만, 월 단위로 계획하거나 의식적으로 소비할 때는 탄소 배출을 줄이려는 노력이 반영되고 있음을 시사합니다.

### 2. 고소비 및 저소비 요약

*   **고소비:** 요식업, 카페, 교통업은 탄소 배출량이 높은 소비 영역입니다.
*   **저소비:** 생활 가전은 탄소 배출량이 낮은 소비 영역입니다.

### 3. 탄소 감축을 위한 동사형 액션 플랜 (5가지)

1.  **대중교통 이용하고, 가까운 거리는 걷거나 자전거 타기.**
2.  **외식 줄이고, 집에서 직접 요